## Библиотеки

In [ ]:
import os
from bs4 import BeautifulSoup as bs
import pandas as pd
import warnings
import random
from datetime import datetime
from lxml import html
import requests
import time

from lxml.etree import ParseError
from lxml.etree import ParserError

import aiohttp
import asyncio
from aiohttp import ClientTimeout
from aiohttp_socks import ProxyConnector
from aiohttp import ClientSession
from stem import Signal
from stem.control import Controller

from tqdm.asyncio import tqdm
from charset_normalizer import from_bytes

from datetime import datetime as Ddtime
from datetime import timedelta

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings("ignore")

In [ ]:
REQUEST_AMOUNT = 10 # больше 10 - плохая идея

semaphore = asyncio.Semaphore(REQUEST_AMOUNT)

### Случайные конфигурации юзера и языков запрашиваемых страниц
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36 Edg/93.0.961.52",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36",
]

LANGUAGES = [
    "en-US,en;q=0.9",
    "en-GB,en;q=0.9",
    "fr-FR,fr;q=0.9",
    "de-DE,de;q=0.9",
    "es-ES,es;q=0.9",
    "ru-RU,ru;q=0.9",
    "zh-CN,zh;q=0.9",
    "ja-JP,ja;q=0.9",
    "it-IT,it;q=0.9",
    "pt-BR,pt;q=0.9",
]

ip_change_lock = asyncio.Lock()

TOR_PROXY = "socks5://127.0.0.1:9150"  # Порт Tor Browser
CONTROL_PORT = 9051  # Порт ControlPort
CONTROL_PASSWORD = "ENTER_YOUR_OWN_PASSWORD"

### Торовский прокси-сервер
proxies = {
    'http': 'socks5://127.0.0.1:9150',
    'https': 'socks5://127.0.0.1:9150'
}

## Функции

In [ ]:
### Функция для обновления IP внутри Тора (используется для
### преодоления 429 ответа от сайтов и прочих неприятностей)
def renew_tor_ip():
    with Controller.from_port(port=CONTROL_PORT) as controller:
        controller.authenticate(password=CONTROL_PASSWORD)
        controller.signal(Signal.NEWNYM)

### Основная функция для асинхронного скачивания HTML-кода страницы
### запрашиваемого ресурса с внедренным обработчиком ошибок
async def fetch(session, url, max_retries=10):
    retries = 0
    while retries < max_retries:
        try:
            async with semaphore:
                await asyncio.sleep(1)  # Задержка между запросами
                headers = {
                    "User-Agent": random.choice(USER_AGENTS),
                    "Accept-Language": random.choice(LANGUAGES),
                }
                async with session.get(url, headers=headers) as response:
                    if response.status == 200 or response.status == 404:
                        content = await response.content.read()
                        en = from_bytes(content)
                        encoding = en.best().encoding
                        return content.decode(encoding)
                    elif response.status == 429:  # Too Many Requests
                        retry_after = response.headers.get("Retry-After")
                        if retry_after:
                            try:
                                wait_time = int(retry_after)
                            except ValueError:
                                retry_time = datetime.strptime(retry_after, "%a, %d %b %Y %H:%M:%S GMT")
                                wait_time = (retry_time - datetime.now()).total_seconds()
                            await asyncio.sleep(wait_time)
                        else:
                            async with ip_change_lock:
                                renew_tor_ip()
                                await asyncio.sleep(15)
                    else:
                        async with ip_change_lock:
                            print(f"Server returned {response.status} for {url}. Changing IP...")
                            renew_tor_ip()
                            await asyncio.sleep(15)
                    retries += 1
        except (aiohttp.ClientError, asyncio.TimeoutError) as e:
            print(f"Error fetching {url}: {e}")
            retries += 1
            if retries < max_retries:
                async with ip_change_lock:
                    print("Changing IP due to error...")
                    renew_tor_ip()
                    await asyncio.sleep(15)
    return None

### Сверка того, является ли извлеченная ссылка
### на модель автомобиля корректно сгенерированной или является
### "синонимом" для корректной (насколько я понимаю логику дрома)
async def process_page(url, session):
    content = await fetch(session, url)
    if content:
        tree = html.fromstring(content)
        title = tree.xpath('//head/title')[0]
        return title.text_content() != "Запрошенная вами страница не существует!"
    return None

### Базовый обработчик для асинхронного запроса
async def worker(url_queue, session, results, pbar):
    while True:
        url = await url_queue.get()
        if url is None:
            break
        try:
            data = await process_page(url, session)
            if data and type(data) != type(None):
                results.append(url)
        except Exception as e:
            print(f"Error processing URL {url}: {e}")
        finally:
            url_queue.task_done()
            pbar.update(1)

### Функция для обраюботки корректных ссылок, извлеченных из скрипта
### основной страницы для машины на сайте Дром
async def extracting_models_url(urls):
    url_queue = asyncio.Queue()
    results = []

    for url in urls:
        url_queue.put_nowait(url)

    connector = ProxyConnector.from_url(TOR_PROXY)
    async with ClientSession(connector=connector) as session:
        with tqdm(total=len(urls), desc=f"Сбор корректных ссылок") as pbar:
            workers = [asyncio.create_task(worker(url_queue, session, results, pbar)) for _ in range(REQUEST_AMOUNT)]
            
            await url_queue.join()

            for _ in range(REQUEST_AMOUNT):
                await url_queue.put(None)

            await asyncio.gather(*workers)

    return results


In [ ]:
### Базовая функция для подсчета количества объявлений
### на запрашиваемой странице. Если таковых там не оказалось
### (а такое бывает, потому что Дром работает как большая 
### таблица из SQL), возвращает нарочито большое значение 10**8
async def adding_amount(url):
    connector = ProxyConnector.from_url(TOR_PROXY)
    async with ClientSession(connector=connector) as session:
        content = await fetch(session, url)
        if content:
            tree = html.fromstring(content)
            if len(tree.xpath('//div[@class="css-1xkq48l eckkbc90"]')) == 0:
                return 10**8
            amo = html.tostring(tree.xpath('//div[@class="css-1xkq48l eckkbc90"]')[0]).decode("utf-8")
            amo = amo[amo.find(">") + 1:].split()[0].replace("&#160;", "")
            return int(amo)
        return None


In [ ]:
def isNumber(s):
    negative = False
    if(s[0] =='-'):
        negative = True
         
    if negative == True:
        s = s[1:]
    try:
        n = int(s)
        return True
    except ValueError:
        return False


In [ ]:
### Функция-экстрактор моделей автомобиля из сырого HTML-кода, который
### на сайте генерирует отдельный .js файлик
def getting_models_for_car(car_name):
    url = f'https://auto.drom.ru/archive/{car_name}/all/'
    response = requests.get(url, proxies = proxies)
    tree = html.fromstring(response.content)

    script_to_read = tree.xpath('//script[@data-drom-module="bulls-list-auto"]')[0]

    script_to_read = html.tostring(script_to_read).decode("utf-8")

    quot = "&quot;"*(script_to_read.find("&quot;") != -1) + '"'*(script_to_read.find("&quot;") == -1)
    fltr = script_to_read.find(quot + "filter" + quot + ":")

    script_to_read = script_to_read[fltr:]
    ending = script_to_read.find("]," + quot + "firmId")
    script_to_read = script_to_read[:ending]
    script_to_read = script_to_read[script_to_read.find("[{") + 1 : script_to_read.rfind("]")]
    script_to_read = script_to_read.split("},{")
    script_to_read = [x.replace(quot, " ") for x in script_to_read]
    script_to_read = [x[x.rfind("["):] for x in script_to_read]

    all_models = []

    for model in (script_to_read):
        probab_list = model[1:-1].split(", ")
        probab_list = [x.strip() for x in probab_list]
        for cd in probab_list:
            if cd.find("\\") == -1 and cd.find(" ") == -1:
                all_models.append(cd)

    all_models = [f'https://auto.drom.ru/archive/{car_name}/{x.replace(" ", "")}/' for x in all_models]
    
    return all_models

### Проверка корректности страницы (рудиментарная функция)
def url_is_ok(url):
    response = requests.get(url, proxies = proxies)
    tree = html.fromstring(response.content)
    title = tree.xpath('//head/title')[0]
    return title.text_content() != "Запрошенная вами страница не существует!"

### Проверка корректности страницы (рудиментарная функция)
def url_is_valid(url):
    response = requests.get(url, proxies = proxies)
    tree = html.fromstring(response.content)
    divs = tree.xpath('//div[@data-ftid="bulls-list_bull"]')

    return len(divs) > 0

### Функция-экстрактор всех доступных поколений и рестайлингов
### конкретной модели автомобиля. Много эвристик, о которых ниже
async def getting_spec_models_for_car(car_url, driver):
    mas = [0, 0]
    generation = 1
    new_url = car_url + f"generation{generation}/restyling{mas[generation]}/"

    all_urls = []
    amo = await adding_amount(car_url)
    if amo >= 10**8:
        return all_urls

    schet = 0

    ### Основная рудиментарная эвристика: перебирать последовательно
    ### поколения и рестайлинги конкретной модели "пока перебирается"
    ### Проблемы: отсутствие асинхронности, медлительность и не гарантированность
    ### результата. Для этого ниже есть более продуманный блок кода
    while url_is_ok(new_url):
        schet += 1
        while url_is_ok(new_url):
            mas[generation] += 1
            new_url = car_url + f"generation{generation}/restyling{mas[generation]}/"
        generation += 1
        mas.append(0)
        new_url = car_url + f"generation{generation}/restyling{mas[generation]}/"
    
    ### Если ничего не сработало в прошлом пункте (почти всегда),
    ### открываем в эмуляторе саму страницу, находим кнопку выбора модели,
    ### нажимаем, и это нам дает возможность скачать опять сырой код страницы,
    ### где уже будут все модели и все их рестайлинги
    if schet <= 1:
        driver.get(car_url)
        time.sleep(3)
        source = driver.page_source
        cla = bs(source, "html.parser").find_all("button")[[x.text for x in bs(source, "html.parser").find_all("button")].index("Поколение")].get("class")[0]
        
        button = driver.find_element(By.CLASS_NAME, cla)
        button.click()

        source = driver.page_source
        # driver.quit()

        tree = html.fromstring(source)
        div = tree.xpath('//div[@data-ga-stats-name="generation_card"]')
        labels = [x.get('aria-label').split(", ") for x in div]
        mas = []
        for L in labels:
            if len(L) == 1:
                mas.append((0, L[0][0]))
            else:
                if isNumber(L[1][0]):
                    mas.append((L[1][0], L[0][0]))
                else:
                    mas.append((1, L[0][0]))
        gen_sty = mas
        for gen_sty in mas:
            generation = int(gen_sty[1])
            restyling = int(gen_sty[0])
            adding_url = car_url + f"generation{generation}/restyling{restyling}"
            all_urls.append(adding_url)
    else:
        mas = [(i, mas[i]) for i in range(len(mas)) if mas[i] > 0]
        if len(mas) == 0:
            mas = [(0, 1)]

        for gen_sty in mas:
            generation = int(gen_sty[0])
            max_sty = int(gen_sty[1])
            for restyling in range(max_sty):
                adding_url = car_url + f"generation{generation}/restyling{restyling}"
                amo = await adding_amount(adding_url) # опять проверка корректности
                if amo < 10**8: all_urls.append(adding_url)

    return all_urls

### Функция-экстрактор всех лет, когда производилась конкретная
### модель в конкретном поколоении и с конкретным рестайлингом
async def getting_spec_years_for_model(model_url, driver = None):
    if driver != None:
        driver.get(model_url)
        page_source = driver.page_source
        tree = html.fromstring(page_source)
    else:
        response = requests.get(model_url, proxies= proxies)
        tree = html.fromstring(response.content)
    exact_name = tree.xpath('//button[@data-ftid="component_select_button"]')[0]
    years = exact_name.text_content().split(", ")[1].replace("н.в.", "2024").split(" - ")
    years = [int(x.strip()) for x in years]
    all_urls = []
    for i in range(years[0], years[1] + 1):
        adding_url = model_url + f"/year-{i}"
        amo = await adding_amount(adding_url) # проверка корректности
        if amo < 10**8: all_urls.append(adding_url)
    
    return all_urls


In [ ]:
colors = [12, 1, 7, 2, 9, 4, 16, 3, 14, 13, 10, 8, 6, 11, 5, 15]

### Команда-воркер для добавления цвета
### запрашиваемого авто
async def color_w(url_queue, session, results, pbar):
    while True:
        url = await url_queue.get()
        if url is None:
            break
        try:
            content = await fetch(session, url)
            if content:
                tree = html.fromstring(content)
                amo = 0
                if len(tree.xpath('//div[@class="css-1xkq48l eckkbc90"]')) == 0:
                    amo = 10**8
                else:
                    amo = html.tostring(tree.xpath('//div[@class="css-1xkq48l eckkbc90"]')[0]).decode("utf-8")
                    amo = amo[amo.find(">") + 1:].split()[0].replace("&#160;", "")
                    amo = int(amo)
            
                if amo > 400:
                    if "order_d" in url:
                        st = url[:url.find("?")]
                        U = [st + "/"*int(st[-1] != "/") + f"?colorid[]={i}&order_d=asc#tabs" for i in colors]
                    else:
                        U = [url + "/"*int(url[-1] != "/") + f"?colorid[]={i}" for i in colors]
                    results.append(U)
                else:
                    results.append([url])
            
        except Exception as e:
            print(f"Error processing URL {url}: {e}")
        finally:
            url_queue.task_done()
            pbar.update(1)

### Сама асинхронная команда
async def color_sratification(urls):
    url_queue = asyncio.Queue()
    results = []

    for url in urls:
        url_queue.put_nowait(url)

    connector = ProxyConnector.from_url(TOR_PROXY)
    async with ClientSession(connector=connector) as session:
        with tqdm(total=len(urls), desc=f"Добавляем стратификацию по цвету") as pbar:
            workers = [asyncio.create_task(color_w(url_queue, session, results, pbar)) for _ in range(REQUEST_AMOUNT)]
            
            await url_queue.join()

            for _ in range(REQUEST_AMOUNT):
                await url_queue.put(None)

            await asyncio.gather(*workers)

    return results


In [ ]:
### Команда-воркер для подсчета количества объявлений по конкретной
### ссылке. Является частью бэк-дора сайта: мы сначала считаем количество
### объявлений по ссылке, а потом начинаем стратифицировать объявления
### по пробегу, указанному в объявлении (сайт опрометчиво дает возможность
### указывать любой пробег с точностью до километра)
async def append_amo(url_queue, session, results, pbar):
    while True:
        url = await url_queue.get()
        if url is None:
            break
        try:
            content = await fetch(session, url)
            if content:
                tree = html.fromstring(content)
                amo = 0
                if len(tree.xpath('//div[@class="css-1xkq48l eckkbc90"]')) == 0:
                    amo = 10**8
                else:
                    amo = html.tostring(tree.xpath('//div[@class="css-1xkq48l eckkbc90"]')[0]).decode("utf-8")
                    amo = amo[amo.find(">") + 1:].split()[0].replace("&#160;", "")
                    amo = int(amo)
            
                results.append((url, amo))
        except Exception as e:
            print(f"Error processing URL {url}: {e}")
        finally:
            url_queue.task_done()
            pbar.update(1)

### Сама асинхронная команда
async def append_amo_for_url_list(urls):
    url_queue = asyncio.Queue()
    results = []

    for url in urls:
        url_queue.put_nowait(url)

    connector = ProxyConnector.from_url(TOR_PROXY)
    async with ClientSession(connector=connector) as session:
        with tqdm(total=len(urls), desc=f"Подсчет количества ссылок по запросу") as pbar:
            workers = [asyncio.create_task(append_amo(url_queue, session, results, pbar)) for _ in range(REQUEST_AMOUNT)]
            
            await url_queue.join()

            for _ in range(REQUEST_AMOUNT):
                await url_queue.put(None)

            await asyncio.gather(*workers)

    return results


In [ ]:
### Команда-воркер для разделения объявлений по указанному
### в них пробегу. Де-факто, это итерация бин-поиска по пробегу
async def stratifier(url_queue, session, results, pbar):
    while True:
        mas = await url_queue.get()
        if type(mas) == type(None):
            break
        url, amo = mas
        try:

            if url == url.replace("probeg", ""):
                x = (url, 0, 1500000, amo)
            else:
                extract = url[url.find("probeg") - 3:]
                extract = extract.split("&")
                probeg = [int(x[x.find("=") + 1:]) for x in extract]

                if "color" in url:
                    if "&min" in url:
                        url_no_probeg = url[:url.find("&min")]
                    else:
                        url_no_probeg = url[:url.find("&max")]
                else:
                    if "/?min" in url:
                        url_no_probeg = url[:url.find("/?min")]
                    else:
                        url_no_probeg = url[:url.find("/?max")]

                if len(probeg) == 1:
                    if url != url.replace("minprobeg", ""):
                        x = (url_no_probeg, probeg[0], 1500000, amo)
                    if url != url.replace("maxprobeg", ""):
                        x = (url_no_probeg, 0, probeg[0], amo)
                else:
                    x = (url_no_probeg, probeg[0], probeg[1], amo)

            new_folds = []
            av_probeg = int((x[1] + x[2])/2)

            if "?" in x[0]:
                mod_url1 = x[0] + f"&minprobeg={x[1]}&maxprobeg={av_probeg}"
            else:
                mod_url1 = x[0] + f"/?minprobeg={x[1]}&maxprobeg={av_probeg}"
            
            content1 = await fetch(session, mod_url1)
            tree1 = html.fromstring(content1)
            amo1 = 0
            if len(tree1.xpath('//div[@class="css-1xkq48l eckkbc90"]')) == 0:
                amo1 = 10**8
            else:
                amo1 = html.tostring(tree1.xpath('//div[@class="css-1xkq48l eckkbc90"]')[0]).decode("utf-8")
                amo1 = amo1[amo1.find(">") + 1:].split()[0].replace("&#160;", "")
                amo1 = int(amo1)

            if amo1 < 10**8:
                f1 = (mod_url1, x[1], max(av_probeg - 1, x[1]), amo1)
                if f1[1] == f1[2] and amo1 > 800:
                    f1[-1] = 10**8
                new_folds.append(f1)
            
            if "?" in x[0]:
                mod_url2 = x[0] + f"&minprobeg={av_probeg}&maxprobeg={x[2]}"
            else:
                mod_url2 = x[0] + f"/?minprobeg={av_probeg}&maxprobeg={x[2]}"

            content2 = await fetch(session, mod_url2)
            tree2 = html.fromstring(content2)
            amo2 = 0
            if len(tree2.xpath('//div[@class="css-1xkq48l eckkbc90"]')) == 0:
                amo2 = 10**8
            else:
                amo2 = html.tostring(tree2.xpath('//div[@class="css-1xkq48l eckkbc90"]')[0]).decode("utf-8")
                amo2 = amo2[amo2.find(">") + 1:].split()[0].replace("&#160;", "")
                amo2 = int(amo2)

            if amo2 < 10**8:
                f2 = (mod_url2, av_probeg, x[2], amo2)
                if f2[1] == f2[2] and amo2 > 800:
                    f2[-1] = 10**8
                new_folds.append(f2)

            appending = [(x[0], x[-1]) for x in new_folds]

            for y in appending:
                results.append(y)

        except Exception as e:
            print(f"Error processing URL {url}: {e}")
        finally:
            url_queue.task_done()
            pbar.update(1)

### Сама асинхронная команда
async def first_iter_of_bin_search(urls_n_amos):
    url_queue = asyncio.Queue()
    results = []

    for mas in urls_n_amos:
        url_queue.put_nowait(mas)

    connector = ProxyConnector.from_url(TOR_PROXY)
    async with ClientSession(connector=connector) as session:
        with tqdm(total=len(urls_n_amos), desc=f"Стратифицируем ссылки по пробегу") as pbar:
            workers = [asyncio.create_task(stratifier(url_queue, session, results, pbar)) for _ in range(REQUEST_AMOUNT)]
            
            await url_queue.join()

            for _ in range(REQUEST_AMOUNT):
                await url_queue.put(None)

            await asyncio.gather(*workers)

    return results


## Код

In [ ]:
### Пример марок для парсинга
cars = """bentley
rolls-royce
Chery
Geely
Honda
Land Rover
Lexus"""

cars = cars.split("\n")
ALL_CARS = [x.lower().strip().replace(" ", "_") for x in cars] # приводим 


all_cars = []
for car_name in tqdm(ALL_CARS, "Оценка количества объявлений для машин"):
    base_url = f'https://auto.drom.ru/archive/{car_name}/all/'
    amo = await adding_amount(base_url)
    all_cars.append([car_name, amo])
all_cars = sorted(all_cars, key = lambda x: x[1])

all_cars

In [ ]:
to_start_with = 0

for j, car_name in enumerate(all_cars[to_start_with:]):

    to_start_with = j

    start_time = time.time()
    car_name, amo = car_name[0], car_name[1]
    print(f"Номер в очереди: {j + 1} из {len(all_cars)}. Парсинг {car_name}. Всего объявлений {amo}")
    base_url = f'https://auto.drom.ru/archive/{car_name}/all/'
    valid_urls = []
    all_models = []
    amo = await adding_amount(base_url)
    if amo <= 400:
        valid_urls.append(base_url)
    else:
        all_models_url = getting_models_for_car(car_name)
        all_models = await extracting_models_url(all_models_url)
    
    PROXY = "socks5://127.0.0.1:9150"
    driver_service = Service(ChromeDriverManager().install())
    chrome_path = "C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"
    chrome_options = webdriver.ChromeOptions()
    chrome_options.binary_location = chrome_path
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument("--blink-settings=imagesEnabled=false")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument(f'--proxy-server={PROXY}')
    driver = webdriver.Chrome(service=driver_service, options=chrome_options)

    for model in tqdm(all_models, desc=f"Собираем уникальные категории каждой модели {car_name}"):
        amo = await adding_amount(model)
        if amo <= 400:
            valid_urls = valid_urls + [model]
        else:
            extra_list = await getting_spec_models_for_car(model, driver)
            for spec_model in extra_list:
                amo = await adding_amount(spec_model)
                if amo <= 400:
                    valid_urls = valid_urls + [spec_model]
                else:
                    try:
                        ex = await getting_spec_years_for_model(spec_model, driver=driver)
                    except (ParseError, ParserError, IndexError):
                        try:
                            ex = await getting_spec_years_for_model(spec_model, driver=driver)
                        except (ParseError, ParserError, IndexError):
                            # print("3")
                            time.sleep(10)
                            ex = await getting_spec_years_for_model(spec_model, driver=driver)
                    valid_urls = valid_urls + ex

    driver.quit()

    href = pd.DataFrame({"Ссылка": valid_urls})
    href.to_excel(f"./cars/{car_name}_href.xlsx", index = False)

    print("="*50, "\n")


In [ ]:
files = []
for filename in os.listdir(r"./cars/"):
    if filename.endswith("href.xlsx"):
        df = pd.read_excel(f"./cars/{filename}")
        files = files + list(df["Ссылка"])

print("Всего ссылок для дальнейшего сбора объявлений:", len(files))

In [ ]:
new_files = await color_sratification(files)
b = new_files[0]
for i in range(1, len(new_files)):
    b = b + new_files[i]
new_files = b

print("Всего ссылок для дальнейшего сбора объявлений (+ разный цвет):",len(new_files))

In [ ]:
### Бэк-ап на всякий случай
href = pd.DataFrame({"Ссылка": new_files})
list_car_name = "_".join([x[0] for x in all_cars])
href.to_excel(f"./cars/{list_car_name}_href.xlsx", index = False)

In [ ]:
href = pd.read_excel(f"./cars/{list_car_name}_href.xlsx")
new_files = list(href["Ссылка"])

In [ ]:
url_n_amo = await append_amo_for_url_list(new_files)

In [ ]:
### Бэк-ап на всякий случай
href = pd.DataFrame({"Ссылка": [x[0] for x in url_n_amo], "Кол-во": [x[1] for x in url_n_amo]})
href.to_excel(f"./cars/{list_car_name}_href.xlsx", index = False)

In [ ]:
href = pd.read_excel(f"./cars/{list_car_name}_href.xlsx")
href = href.to_dict("list")
url_n_amo = [(href["Ссылка"][i], href["Кол-во"][i]) for i in range(len(href["Ссылка"]))]
new_files = [url_n_amo[0]]

for i in range(1, len(url_n_amo)):
    if url_n_amo[i][1] < 10**8:
        new_files.append(url_n_amo[i])

final = []
remain = []

for i in range(len(new_files)):
    x = new_files[i]
    if x[1] <= 400:
        final.append(x)
    elif x[1] <= 800:
        final.append((x[0], x[1]//2))
        inverse = x[0] + "/?"*int(x[0].find("?") == -1) + "&"*int(x[0].find("?") != -1) + "order_d=asc#tabs"
        final.append((inverse, x[1]//2))
    else:
        remain.append(x)

print("Готовых ссылок:", len(final))
print("Ссылок для доработки:", len(remain))

In [ ]:
problems = []
### Фактический, бинарный поиск
while len(remain) > 10:
    print("Готовых ссылок:", len(final))
    print("Ссылок для доработки:", len(remain))
    print("Ссылок с критической проблемой:", len(problems))
    n_f = await first_iter_of_bin_search(remain)
    remain = []
    for i in range(len(n_f)):
        x = n_f[i]
        if x[1] <= 400:
            final.append(x)
        elif x[1] <= 800:
            final.append((x[0], x[1]//2))
            inverse = x[0] + "/?"*int(x[0].find("?") == -1) + "&"*int(x[0].find("?") != -1) + "order_d=asc#tabs"
            final.append((inverse, x[1]//2))
        elif x[1] == 10**8:
            problems.append(x)
        else:
            remain.append(x)
    print("="*50, "\n")

In [ ]:
### Отсмотр глазами "неудачных" ссылок, которые нужно доделить руками.
### Христоматийный пример: Лад Приор первого поколения 2008 года производства черного цвета
### с пробегом 200 тысяч км (https://auto.drom.ru/archive/lada/priora/generation1/restyling0/year-2008/?colorid[]=1&minprobeg=200000&maxprobeg=200000)
### больше 800 (854 на 29.01.2025) => наблюдения "посередине" никак не достатать,
### отчего код сверху зациклится до бесконечности, если будет "while True" в начале
remain

In [ ]:
### Пример доработки руками!!!

# n_f = []

# n_f.append(
#     ("https://auto.drom.ru/archive/honda/accord/generation8/restyling0/year-2008/?colorid[]=1&minprobeg=164062&maxprobeg=180000",
#      629)
# )

# n_f.append(
#     ("https://auto.drom.ru/archive/honda/accord/generation8/restyling0/year-2008/?colorid[]=1&minprobeg=180000&maxprobeg=187500",
#      179)
# )

# n_f.append(
#     ("https://auto.drom.ru/archive/honda/accord/generation8/restyling0/year-2008/?colorid[]=1&minprobeg=187500&maxprobeg=200000",
#      529)
# )

# n_f.append(
#     ("https://auto.drom.ru/archive/honda/accord/generation8/restyling0/year-2008/?colorid[]=1&minprobeg=200001&maxprobeg=210937",
#      278)
# )

# n_f.append(
#     ("https://auto.drom.ru/archive/geely/monjaro/generation1/restyling0/year-2023/new/?colorid[]=3",
#      565)
# )

# n_f.append(
#     ("https://auto.drom.ru/archive/geely/monjaro/generation1/restyling0/year-2023/used/?colorid[]=3&maxprobeg=23437",
#      387)
# )

### Самый плохой случай, где мы потеряли 54 наблюдения...
# n_f.append(
#     ("https://auto.drom.ru/archive/lada/priora/generation1/restyling0/year-2008/?colorid[]=1&minprobeg=200000&maxprobeg=200000",
#      800)
# )

# for i in range(len(n_f)):
#     x = n_f[i]
#     if x[1] <= 400:
#         final.append(x)
#     elif x[1] <= 800:
#         final.append((x[0], x[1]//2))
#         inverse = x[0] + "/?"*int(x[0].find("?") == -1) + "&"*int(x[0].find("?") != -1) + "order_d=asc#tabs"
#         final.append((inverse, x[1]//2))

In [ ]:
print("Всего объявлений для парсинга:", sum([x[1] for x in final]))

In [ ]:
### Пример разделения ссылок для первичного парсинга

i = 0
new_files = [x[0] for x in final]
while i < len(new_files):
    ongoing = new_files[i:min(i + 37, len(new_files))]
    pd.DataFrame({"Ссылка": ongoing}).to_excel(f"./cars/href/{i//37}car.xlsx", index = False)
    i += 37